In [65]:
import cv2
import numpy as np
import os
import pandas as pd
import mediapipe as mp
data_path = r"C:\Users\Rohit Varre\Desktop\MliS\Python\SYN data" 

In [66]:
# Mediapipe methods to extract and display hand cooridnates
mp_drawing = mp.solutions.drawing_utils     # Displays the key hand points
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    min_detection_confidence=0.4,
    max_num_hands=1,
    min_tracking_confidence=0.4
) 

In [67]:
def store_images(ges_id):
    """Stores gesture images with the gesture title - ges_id as folder name"""
    total_pics=300            # Total number of data points collected
    cap = cv2.VideoCapture(0) # Opencv command to access the webcamera
    data_path = r"C:\Users\Rohit Varre\Desktop\MliS\Python\SYN data" 
    folder_path = os.path.join(data_path,str(ges_id))
    pic_no = 0                                        # Running count of the number of pics collected
    flag_start_capturing = False 
    
    # loops through the image frames during the video
    while True:
        ret,frame = cap.read()       
        image = cv2.flip(frame,1)                     # To deal with image mirroring
        
        # Cooridnates
        x = []
        y = []
        z = []
        
        # Displays the running count of the number of pics collected
        cv2.putText(image, str(pic_no), (30,400), cv2.FONT_HERSHEY_TRIPLEX, 2.5, (127,127,255))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Converting BGR to RGB as mediapipe works in RGB
        image.flags.writeable = True
        results = hands.process(image)                 # Mediapipe command to find hand cooridnates
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Converting RGB to BGR - default OpenCV format
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                # Extracting the coordinates of all the key points
                for i in hand_landmarks.landmark:
                    x.append(i.x)
                    y.append(i.y)
                    z.append(i.z)
        cv2.imshow('MediaPipe Hands', image)
        
        keypress = cv2.waitKey(5)
        if keypress == 32:                        # Click space to start or pause capturing the datapoints
            if flag_start_capturing == False:
                flag_start_capturing = True
            else:
                flag_start_capturing = False
        if flag_start_capturing == True and results.multi_hand_landmarks:
            df = pd.DataFrame({'x':x,'y':y,'z':z})
            df.to_csv(r"C:\Users\Rohit Varre\Desktop\MliS\Python\SYN data/" + str(ges_id)  + '/' + str(pic_no) + '.csv')
            pic_no = pic_no+1
        if pic_no == total_pics:
            break
        if keypress == 27:                       # Click escape key to stop the process
            break
    
    cv2.destroyAllWindows()
    cap.release()

In [84]:
# Driver code to create a folder with gesture title as folder name
print("Gesture Title:")
temp = input()
temp = temp.lower().lstrip().rstrip()

Gesture Title:
w


In [85]:
# Create a folder with the above name if it doesn't exist.
folder_path = folder_path=os.path.join(data_path,temp)
if not os.path.exists(folder_path):  
    os.mkdir(folder_path)
store_images(temp)